# MONITOREO DE PRECIOS DE CRIPTOMONEDAS A TRAVÉS DE APIs CON ANÁLISIS DE TENDENCIAS

## EXTRACCION DE DATOS

### Primera API: coingecko

In [11]:
import requests
import pandas as pd

# URL de la API para obtener monedas con sus datos relevantes
url = "https://api.coingecko.com/api/v3/coins/markets"

# Parámetros de la solicitud
params = {
    'vs_currency': 'usd',   # Monedas en USD
    'order': 'market_cap_desc',  # Ordenar por capitalización
    'per_page': 100,        # Cantidad de monedas por página
    'sparkline': False      # Información de gráficos no solicitada
}

# Función para obtener todas las monedas con datos completos
def obtener_datos_moneda(total_monedas_requeridas, parametros_requeridos):
    monedas_con_datos = []
    page = 1
    while len(monedas_con_datos) < total_monedas_requeridas:
        params['page'] = page
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            monedas_filtradas = filtrar_datos_completos(data, parametros_requeridos)
            monedas_con_datos.extend(monedas_filtradas)
            page += 1
        else:
            print("Error en la solicitud:", response.status_code)
            print(response.json())
            break
    return monedas_con_datos

# Función para filtrar monedas con datos completos
def filtrar_datos_completos(monedas, parametros_requeridos):
    monedas_con_datos = []
    for moneda in monedas:
        tiene_datos_completos = all(moneda[param] is not None for param in parametros_requeridos)
        if tiene_datos_completos:
            monedas_con_datos.append(moneda)
    return monedas_con_datos

# Parámetros requeridos
parametros_requeridos = ['current_price', 'total_volume', 'market_cap', 'circulating_supply', 'total_supply', 'max_supply', 'ath_date', 'atl_date']

# Obtener las monedas con datos completos
total_monedas_requeridas = 500
monedas_filtradas = obtener_datos_moneda(total_monedas_requeridas, parametros_requeridos)

# Convertir a DataFrame
df_monedas = pd.DataFrame(monedas_filtradas)

# Imprimir la cantidad de datos extraídos
print(f"Datos extraídos: {len(df_monedas)} monedas con datos completos")

Error en la solicitud: 429
{'status': {'error_code': 429, 'error_message': "You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits."}}
Datos extraídos: 315 monedas con datos completos


In [12]:
# Mostrar las columnas para CoinGecko
print(df_monedas.columns)

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')


El DataFrame "df_monedas" contendrá las columnas:
- id (identificador único de la moneda).
- symbol (símbolo de la moneda, por ejemplo, "BTC").
- name (nombre de la moneda, por ejemplo, "Bitcoin").
- image (URL del logo de la moneda).
- current_price (precio actual en USD).
- market_cap (capitalización de mercado en USD).
- market_cap_rank (rango según la capitalización de mercado).
- fully_diluted_valuation (valorización completamente diluida en USD).
- total_volume (volumen total de comercio en las últimas 24 horas).
- high_24h (precio más alto alcanzado en las últimas 24 horas).
- low_24h (precio más bajo alcanzado en las últimas 24 horas).
- price_change_24h (cambio en el precio en las últimas 24 horas).
- price_change_percentage_24h (porcentaje de cambio en el precio en las últimas 24 horas).
- market_cap_change_24h (cambio en la capitalización de mercado en las últimas 24 horas).
- market_cap_change_percentage_24h (porcentaje de cambio en la capitalización de mercado en las últimas 24 horas).
- circulating_supply (suministro circulante actual de la moneda).
- total_supply (suministro total de la moneda).
- max_supply (suministro máximo de la moneda).
- ath (precio más alto alcanzado en toda la historia).
- ath_change_percentage (porcentaje de cambio respecto al precio ATH).
- ath_date (fecha en que se alcanzó el precio ATH).
- atl (precio más bajo alcanzado en toda la historia).
- atl_change_percentage (porcentaje de cambio respecto al precio ATL).
- atl_date (fecha en que se alcanzó el precio ATL).
- roi (retorno de inversión en porcentaje).
- last_updated (última fecha de actualización de los datos).

### Segunda API: coingecko

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '64bb4596-fe19-4b50-8048-ee8892f1f527',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)
  

In [ ]:
# Crear una lista de diccionarios con la información relevante de cada criptomoneda
cryptos_data = []

for crypto in data['data']:
    cryptos_data.append({
        'Name': crypto['name'],
        'Symbol': crypto['symbol'],
        'Price (USD)': crypto['quote']['USD']['price'],
        'Market Cap (USD)': crypto['quote']['USD']['market_cap'],
        'Volume 24h (USD)': crypto['quote']['USD']['volume_24h'],
        'Percent Change 1h (%)': crypto['quote']['USD']['percent_change_1h'],
        'Percent Change 24h (%)': crypto['quote']['USD']['percent_change_24h'],
        'Percent Change 7d (%)': crypto['quote']['USD']['percent_change_7d'],
        'Circulating Supply': crypto['circulating_supply'],
        'Total Supply': crypto['total_supply'],
        'Max Supply': crypto.get('max_supply', None),  # Puede no estar presente
        'Last Updated': crypto['last_updated']
    })

# Crear el DataFrame
df = pd.DataFrame(cryptos_data)

# Mostrar las primeras filas del DataFrame
print(df.head())

# Para ver todas las variables del dataframe
print(df.columns)

### Tercera API: CryptoCompare

In [1]:
import requests

response = requests.get(
    "https://min-api.cryptocompare.com/data/v2/news/",
    params={"lang":"EN","api_key":"76e0850532f6a9e0faa1926f9e4af3ceb15224cfb75e8f8638ab876bfa7547ad"},
    headers={"Content-type":"application/json; charset=UTF-8"}
)

json_response = response.json()
print(json_response)

{'Type': 100, 'Message': 'News list successfully returned', 'Promoted': [], 'Data': [{'id': '37824472', 'guid': 'https://www.newsbtc.com/?p=664794', 'published_on': 1734578337, 'imageurl': 'https://images.cryptocompare.com/news/default/newsbtc.png', 'title': 'Ethereum Price Falls 5%: Correction or Start of a Bigger Slide?', 'url': 'https://www.newsbtc.com/analysis/eth/ethereum-price-falls-3550/', 'body': 'Ethereum price started a sharp decline below the $3,880 zone. ETH is down over 5% and is showing bearish signs below the $3,680 level. Ethereum started a fresh decline below the $3,800 zone. The price is trading below $3,680 and the 100-hourly Simple Moving Average. There is a new connecting bearish trend line forming with resistance at $3,800 on the hourly chart of ETH/USD (data feed via Kraken). The pair could continue to move down if it fails to stay above the $3,550 support zone. Ethereum Price Dives Below $3,800 Ethereum price struggled to stay above the $3,880 level and started 

In [3]:
import pandas as pd
# Convertir a DataFrame si hay datos
if "Data" in json_response:
    df = pd.DataFrame(json_response["Data"])

    # Seleccionar solo columnas importantes
    df = df[["title", "source", "published_on", "url", "body"]]
    print(df.head())
else:
    print("No hay datos para mostrar.")

                                               title      source  \
0  Ethereum Price Falls 5%: Correction or Start o...     newsbtc   
1  WazirX Faces Decline Due to Binance’s Delistin...  cointurken   
2  BlackRock’s ETHA Witnesses Daily Inflow of +$8...    coinotag   
3  BlackRock IBIT Reports Record Daily Net BTC In...    coinotag   
4  Japan Bank Aligns with Market Predictions, Set...    coinotag   

   published_on                                                url  \
0    1734578337  https://www.newsbtc.com/analysis/eth/ethereum-...   
1    1734578083  https://en.coin-turk.com/wazirx-faces-decline-...   
2    1734577516  https://en.coinotag.com/breakingnews/blackrock...   
3    1734577291  https://en.coinotag.com/breakingnews/blackrock...   
4    1734577248  https://en.coinotag.com/breakingnews/japan-ban...   

                                                body  
0  Ethereum price started a sharp decline below t...  
1  WazirX's price dropped significantly due to Bi...  
2  Bl